In [8]:
import copy

from src.utils import get_project_root

xamthines = {
    "xanthine": "C1=NC2=C(N1)C(=O)NC(=O)N2",
    "caffeine": "CN1C=NC2=C1C(=O)N(C(=O)N2C)C",
    "pentoxifylline": "CC(=O)CCCCN1C(=O)C2=C(N=CN2C)N(C1=O)C",
    "dyphylline": "CN1C2=C(C(=O)N(C1=O)C)N(C=N2)CC(CO)O",
    "IBMX": "CC(C)CN1C2=C(C(=O)N(C1=O)C)NC=N2",
    "ENPROFYLLINE": "CCCN1C2=C(C(=O)NC1=O)NC=N2",
    "Xanthosine": "C1=NC2=C(N1[C@H]3[C@@H]([C@@H]([C@H](O3)CO)O)O)NC(=O)NC2=O",
    "uracil": "C1=CNC(=O)NC1=O",
    "uridine": "C1=CN(C(=O)NC1=O)[C@H]2[C@@H]([C@@H]([C@H](O2)CO)O)O",
    "oxypurinol": "C1=NNC2=C1C(=O)NC(=O)N2",
    "allopurinol": "C1=NNC2=C1C(=O)NC=N2",
    "2_4-dioxo-1H_2H_3H_4H-pyrido_2_3-d_pyrimidine-6-carboxylic_acid": "O=C1NC(C2=C(N1)N=CC(=C2)C(=O)O)=O",
}

In [9]:
import yaml

base_file = get_project_root() / "inputs/ryr1_rabit_atp_pocket.yaml"

with open(base_file, "r") as f:
    base_data = yaml.safe_load(f)

base_data

{'constraints': [{'pocket': {'binder': 'B',
    'contacts': [['A', 675], ['A', 3], ['A', 934]],
    'max_distance': 5}}],
 'properties': [{'affinity': {'binder': 'B'}}],
 'sequences': [{'protein': {'id': 'A',
    'msa': 'inputs/msa/ryr1_rabit.a3m',
    'sequence': 'EVSFEEKEMEKQRLLYQQSRLHTRGAAEMVLQMISACKGETGAMVSSTLKLGISILNGGNAEVQQKMLDYLKDKKEVGFFQSIQALMQTCSVLDLNAFERQNKAEGLGMVNEDGTVINRQNGEKVMADDEFTQDLFRFLQLLCEGHNNDFQNYLRTQTGNTTTINIIICTVDYLLRLQESISDFYWYYSGKDVIEEQGKRNFSKAMSVAKQVFNSLTEYIQGPCTGNQQSLAHSRLWDAVVGFLHVFAHMMMKLAQDSSQIELLKELLDLQKDMVVMLLSLLEGNVVNGMIARQMVDMLVESSSNVEMILKFFDMFLKLKDIVGSEAFQDYVTDPRGLISKKDFQKAMDSQKQFTGPEIQFLLSCSEADENEMINFEEFANRFQEPARDIGFNVAVLLTNLSEHVPHDPRLRNFLELAESILEYFRPYLGRIEIMGASRRIERIYFEISETNRAQWEMPQVKESKRQFIFDVVNEGGEAEKMELFVSFCEDTIFEMQIAAWGELEVQRVKFLNYLSRNFYTLRFLALFLAFAINFILLFYKVSDSPPGEDDMEGSAAGDLAGAGSGGGSGWGSGAGEEAEGDEDENMVYYFLEESTGYMEPALWCLSLLHTLVAFLCIIGYNCLKVPLVIFKREKELARKLEFDGLYITEQPGDDDVKGQWDRLVLNTPSFPSNYWDKFVKRKVLDKHGDIFGRERIAELLGMDLASLEITAHNERKPDPPPGLLTWLMSIDVK

In [10]:
output_dir = get_project_root() / "inputs/xanthines"

if not output_dir.exists():
    print(f"Creating output directory {output_dir}")
    output_dir.mkdir(parents=True, exist_ok=True)


def add_water(
    data: dict,
    max_ligand_id: str = "B",
):
    new_data = copy.deepcopy(data)

    new_ligand_id = chr(ord(max_ligand_id) + 1)

    new_data["sequences"].append(
        {
            "ligand": {
                "id": new_ligand_id,
                "smiles": "O",
            }
        }
    )

    # force the water into the pocket
    pocket_org = copy.deepcopy(new_data["constraints"][0])
    pocket_org["pocket"]["binder"] = "C"
    pocket_org["pocket"]["max_distance"] = pocket_org["pocket"]["max_distance"] * 1.25
    new_data["constraints"].append(pocket_org)
    return new_data


def add_atp(
    data: dict,
    max_ligand_id: str = "B",
):
    new_data = copy.deepcopy(data)

    new_ligand_id = chr(ord(max_ligand_id) + 1)

    new_data["sequences"].append({"ligand": {"id": new_ligand_id, "ccd": "ATP"}})


for ligand, smiles in xamthines.items():
    with_new_ligand = copy.deepcopy(base_data)

    for sequences in with_new_ligand["sequences"]:
        if "ligand" not in sequences:
            continue

        if "ccd" in sequences["ligand"] and sequences["ligand"]["ccd"] != "CFF":
            print(f"Skipping {sequences['ligand']['ccd']} as its not caffeine")
            continue

        del sequences["ligand"]["ccd"]

        sequences["ligand"]["smiles"] = smiles

    print(with_new_ligand)

    output_file = output_dir / f"{ligand}.yaml"

    with open(output_file, "w") as f:
        f.write(yaml.dump(with_new_ligand, default_flow_style=False, sort_keys=False))

    print(f"Saved {ligand} to {output_file}")

    # # add water
    # data_with_water = add_water(with_new_ligand)
    #
    # with open(output_dir / f"{ligand}-with-water.yaml", "w") as f:
    #     f.write(yaml.dump(data_with_water, default_flow_style=False, sort_keys=False))
    # print(f"Saved {ligand} with water to {output_dir / f'with-water-{ligand}.yaml'}")

{'constraints': [{'pocket': {'binder': 'B', 'contacts': [['A', 675], ['A', 3], ['A', 934]], 'max_distance': 5}}], 'properties': [{'affinity': {'binder': 'B'}}], 'sequences': [{'protein': {'id': 'A', 'msa': 'inputs/msa/ryr1_rabit.a3m', 'sequence': 'EVSFEEKEMEKQRLLYQQSRLHTRGAAEMVLQMISACKGETGAMVSSTLKLGISILNGGNAEVQQKMLDYLKDKKEVGFFQSIQALMQTCSVLDLNAFERQNKAEGLGMVNEDGTVINRQNGEKVMADDEFTQDLFRFLQLLCEGHNNDFQNYLRTQTGNTTTINIIICTVDYLLRLQESISDFYWYYSGKDVIEEQGKRNFSKAMSVAKQVFNSLTEYIQGPCTGNQQSLAHSRLWDAVVGFLHVFAHMMMKLAQDSSQIELLKELLDLQKDMVVMLLSLLEGNVVNGMIARQMVDMLVESSSNVEMILKFFDMFLKLKDIVGSEAFQDYVTDPRGLISKKDFQKAMDSQKQFTGPEIQFLLSCSEADENEMINFEEFANRFQEPARDIGFNVAVLLTNLSEHVPHDPRLRNFLELAESILEYFRPYLGRIEIMGASRRIERIYFEISETNRAQWEMPQVKESKRQFIFDVVNEGGEAEKMELFVSFCEDTIFEMQIAAWGELEVQRVKFLNYLSRNFYTLRFLALFLAFAINFILLFYKVSDSPPGEDDMEGSAAGDLAGAGSGGGSGWGSGAGEEAEGDEDENMVYYFLEESTGYMEPALWCLSLLHTLVAFLCIIGYNCLKVPLVIFKREKELARKLEFDGLYITEQPGDDDVKGQWDRLVLNTPSFPSNYWDKFVKRKVLDKHGDIFGRERIAELLGMDLASLEITAHNERKPDPPPGLLTWLMSIDVKYQIWKFGVIFTDNSFLYL